# Data access

## CSV files

There's multiple functions in the `readr` for CSV file reading. Let's use them on a dataset available in Kaggle that has homemade beer recipes from Brewer's Friend [[1]](https://www.kaggle.com/jtrofe/beer-recipes).

Let's check the first few lines of the data with base R's `file` and `readLines`.

In [1]:
example_file <- file("/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv",'r')
first_lines <- readLines(example_file,n=2)
close(example_file)

print(first_lines)

[1] "BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount"    
[2] "1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70,N/A,Specific Gravity,All Grain,N/A,17.78,corn sugar,4.5 oz"


Before we choose which reader we want to use we need to check the format of the data. 

`readr` has predefined functions for the following data formats [[1]](http://readr.tidyverse.org/reference/read_delim.html):
- `read_delim` parses generic data delimited by a character
- `read_csv` assumes that the data is delimited by commas
- `read_csv2` assumes that the data is delimited by semicolons
- `read_tsv` assumes that the data is delimited by tabs

In this case we want to use `read_csv`. Do note that we limit ourselves to first 100 values for faster parsing.

In [2]:
library(tidyverse)

beer_recipes <- read_csv("/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv", n_max=100)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Parsed with column specification:
cols(
  .default = col_character(),
  BeerID = col_double(),
  StyleID = col_double(),
  `Size(L)` = col_double(),
  OG = col_double(),
  FG = col_double(),
  ABV = col_double(),
  IBU = col_double(),
  Color = col_double(),
  BoilSize = col_double(),
  BoilTime = col_double(),
  Efficiency = col_double()
)

See spec(...) for full column specifications.



From the output one can see that `read_csv` tries to parse the datatype of the column automatically.

By running `spec` one can see the full definitons.

In [3]:
spec(beer_recipes)

cols(
  BeerID = col_double(),
  Name = col_character(),
  URL = col_character(),
  Style = col_character(),
  StyleID = col_double(),
  `Size(L)` = col_double(),
  OG = col_double(),
  FG = col_double(),
  ABV = col_double(),
  IBU = col_double(),
  Color = col_double(),
  BoilSize = col_double(),
  BoilTime = col_double(),
  BoilGravity = col_character(),
  Efficiency = col_double(),
  MashThickness = col_character(),
  SugarScale = col_character(),
  BrewMethod = col_character(),
  PitchRate = col_character(),
  PrimaryTemp = col_character(),
  PrimingMethod = col_character(),
  PrimingAmount = col_character()
)

Many of the data columns seem to be characters instead of numbers. Let's use `col_types`-argument to specify a better definition.

In [4]:

beer_recipes <- read_csv("/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv",
    col_types=cols(
      BeerID = col_integer(),
      Name = col_character(),
      URL = col_character(),
      Style = col_character(),
      StyleID = col_integer(),
      `Size(L)` = col_double(),
      OG = col_double(),
      FG = col_double(),
      ABV = col_double(),
      IBU = col_double(),
      Color = col_double(),
      BoilSize = col_double(),
      BoilTime = col_double(),
      BoilGravity = col_double(),
      Efficiency = col_double(),
      MashThickness = col_double(),
      SugarScale = col_character(),
      BrewMethod = col_character(),
      PitchRate = col_double(),
      PrimaryTemp = col_double(),
      PrimingMethod = col_character(),
      PrimingAmount = col_character()
    ),
    n_max=100
)

Warning message:
“208 parsing failures.
row           col expected actual                                                                      file
  1 MashThickness a double    N/A '/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
  1 PitchRate     a double    N/A '/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
  2 MashThickness a double    N/A '/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
  2 PitchRate     a double    N/A '/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
  2 PrimaryTemp   a double    N/A '/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
... ............. ........ ...... .........................................................................
See problems(...) for more details.
”


This produced a lot of problems. Let's check the problems with `problems`.

In [5]:
problems(beer_recipes)

row,col,expected,actual,file
<int>,<chr>,<chr>,<chr>,<chr>
1,MashThickness,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
1,PitchRate,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
2,MashThickness,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
2,PitchRate,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
2,PrimaryTemp,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
3,BoilGravity,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
3,MashThickness,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
3,PitchRate,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'
3,PrimaryTemp,a double,N/A,'/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv'


Most of the problems seem to be related to _N/A_ not being a recognized name for `NA`. Let's add that to the initial read call with `na`-argument. 

In [6]:
beer_recipes <- read_csv("/m/jhnas/jupyter/shareddata/python-r-data-analysis/beers/recipeData.csv",na=c("","NA","N/A"), n_max=100)

spec(beer_recipes)

Parsed with column specification:
cols(
  .default = col_double(),
  Name = col_character(),
  URL = col_character(),
  Style = col_character(),
  SugarScale = col_character(),
  BrewMethod = col_character(),
  PrimingMethod = col_character(),
  PrimingAmount = col_character()
)

See spec(...) for full column specifications.



cols(
  BeerID = col_double(),
  Name = col_character(),
  URL = col_character(),
  Style = col_character(),
  StyleID = col_double(),
  `Size(L)` = col_double(),
  OG = col_double(),
  FG = col_double(),
  ABV = col_double(),
  IBU = col_double(),
  Color = col_double(),
  BoilSize = col_double(),
  BoilTime = col_double(),
  BoilGravity = col_double(),
  Efficiency = col_double(),
  MashThickness = col_double(),
  SugarScale = col_character(),
  BrewMethod = col_character(),
  PitchRate = col_double(),
  PrimaryTemp = col_double(),
  PrimingMethod = col_character(),
  PrimingAmount = col_character()
)

Now most of the columns seem correct. Last column seems to include units (_oz_). Using mutate is probably easiest way of getting rid of them.

Let' use `str_remove` to remove it  [[str_remove]](https://stringr.tidyverse.org/reference/str_remove.html).

After that we can convert the column to double and use `str` to check that our dataset looks fine.

In [7]:
beer_recipes <- beer_recipes %>%
    mutate(PrimingAmount=as.double(str_remove(PrimingAmount, ' oz')))

str(beer_recipes)

Warning message in rlang::eval_tidy(~as.double(str_remove(PrimingAmount, " oz")), :
“NAs introduced by coercion”


Classes ‘spec_tbl_df’, ‘tbl_df’, ‘tbl’ and 'data.frame':	100 obs. of  22 variables:
 $ BeerID       : num  1 2 3 4 5 6 7 8 9 10 ...
 $ Name         : chr  "Vanilla Cream Ale" "Southern Tier Pumking clone" "Zombie Dust Clone - EXTRACT" "Zombie Dust Clone - ALL GRAIN" ...
 $ URL          : chr  "/homebrew/recipe/view/1633/vanilla-cream-ale" "/homebrew/recipe/view/16367/southern-tier-pumking-clone" "/homebrew/recipe/view/5920/zombie-dust-clone-extract" "/homebrew/recipe/view/5916/zombie-dust-clone-all-grain" ...
 $ Style        : chr  "Cream Ale" "Holiday/Winter Special Spiced Beer" "American IPA" "American IPA" ...
 $ StyleID      : num  45 85 7 7 20 10 86 45 129 86 ...
 $ Size(L)      : num  21.8 20.8 18.9 22.7 50 ...
 $ OG           : num  1.05 1.08 1.06 1.06 1.06 ...
 $ FG           : num  1.01 1.02 1.02 1.02 1.01 ...
 $ ABV          : num  5.48 8.16 5.91 5.8 6.48 5.58 7.09 5.36 5.77 8.22 ...
 $ IBU          : num  17.6 60.6 59.2 54.5 17.8 ...
 $ Color        : num  4.83 15.64 8.98 8.

Let's say that we want to write the resulting `tibble` in a format that is easily readable in Excel.

For this we'd want to use `write_excel_csv` (there are similar functions for normal csv, tsv etc.) [[write_excel_csv]](http://readr.tidyverse.org/reference/write_delim.html).  

In [8]:
write_excel_csv(beer_recipes, 'beer-recipes-excel-format.csv')

# Feather

Let's say you have a big dataset you have pre-processed with R, but want to analyze with Python. The new feather-format that uses Apache Arrow's data specification is created by the creators of Tidy-R and Pandas and it should be interoprable with both of them [[feather's page in Github]](https://github.com/wesm/feather).

What matters the most is that it is fast and compact (because it is a binary data format).

Using it is simple, just load `feather`-library an write data with `write_feather` [[write_feather]](https://cran.r-project.org/web/packages/feather/feather.pdf).

Loading data is done with `read_feather`.

Do note that more complex structures like nested tibbles do not necessarily fit into a feather.

Let's install `feather`:

In [9]:
if (!file.exists('rlibs')) {
    dir.create('rlibs')
}
if (!file.exists('rlibs/feather')) {
    install.packages('feather', repos="http://cran.r-project.org", lib='rlibs')
}
library(feather, lib.loc='rlibs')

In [10]:
write_feather(beer_recipes,'beer_recipes.feather')

beer_recipes2 <- read_feather('beer_recipes.feather')

# Database access

There exists a package `DBI` that defines a common interface that can be used to access various different databases [[DBI]](https://dbi.r-dbi.org/).

When using `DBI`, one can also use `dbplyr` to run `tidyverse` verbs (`select`, `map`, etc.) to database queries without loading the whole database into memory.

In [11]:
if (!file.exists('rlibs')) {
    dir.create('rlibs')
}
if (!file.exists('rlibs/DBI')) {
    install.packages('DBI', repos="http://cran.r-project.org", lib='rlibs')
}
if (!file.exists('rlibs/dbplyr')) {
    install.packages('dbplyr', repos="http://cran.r-project.org", lib='rlibs')
}
library(DBI, lib.loc='rlibs')
library(dbplyr, lib.loc='rlibs')


Attaching package: ‘dbplyr’


The following objects are masked from ‘package:dplyr’:

    ident, sql




Let's use `tbl_memdb` from `dbplyr` to add `beer_recipes` to a temporary in-memory database [[tbl_memdb]](https://dbplyr.tidyverse.org/reference/memdb_frame.html).

In [12]:
beers_db <- tbl_memdb(beer_recipes)

The returned object is a SQL table that acts like a tibble.

In [13]:
print(beers_db)
beers_db %>%
    select(Name, ABV, OG, FG)

# Source:   table<beer_recipes> [?? x 22]
# Database: sqlite 3.22.0 [:memory:]
   BeerID Name  URL   Style StyleID `Size(L)`    OG    FG   ABV   IBU Color
    <dbl> <chr> <chr> <chr>   <dbl>     <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1      1 Vani… /hom… Crea…      45      21.8  1.06  1.01  5.48  17.6  4.83
 2      2 Sout… /hom… Holi…      85      20.8  1.08  1.02  8.16  60.6 15.6 
 3      3 Zomb… /hom… Amer…       7      18.9  1.06  1.02  5.91  59.2  8.98
 4      4 Zomb… /hom… Amer…       7      22.7  1.06  1.02  5.8   54.5  8.5 
 5      5 Bakk… /hom… Belg…      20      50    1.06  1.01  6.48  17.8  4.57
 6      6 Sier… /hom… Amer…      10      24.6  1.06  1.01  5.58  40.1  8   
 7      7 Russ… /hom… Impe…      86      22.7  1.07  1.02  7.09 269.   6.33
 8      8 Spot… /hom… Crea…      45      20.8  1.05  1.01  5.36  20.0  5.94
 9      9 Choc… /hom… Robu…     129      22.7  1.06  1.02  5.77  31.6 34.8 
10     10 Mang… /hom… Impe…      86      20.8  1.08  1.02  8.22  93.0  8.29
# … with 

# Source:   lazy query [?? x 4]
# Database: sqlite 3.22.0 [:memory:]
   Name                                           ABV    OG    FG
   <chr>                                        <dbl> <dbl> <dbl>
 1 Vanilla Cream Ale                             5.48  1.06  1.01
 2 Southern Tier Pumking clone                   8.16  1.08  1.02
 3 Zombie Dust Clone - EXTRACT                   5.91  1.06  1.02
 4 Zombie Dust Clone - ALL GRAIN                 5.8   1.06  1.02
 5 Bakke Brygg Belgisk Blonde 50 l               6.48  1.06  1.01
 6 Sierra Nevada Pale Ale Clone                  5.58  1.06  1.01
 7 Russian River Pliny the Elder (original)      7.09  1.07  1.02
 8 Spotted Clown (New Glarus Spotted Cow clone)  5.36  1.05  1.01
 9 Chocolate Vanilla Porter                      5.77  1.06  1.02
10 Mango Habanero IPA                            8.22  1.08  1.02
# … with more rows

The difference between using normal tibbles and this database connection is that database queries are all connected and only evaluated when data is requsted. One can see the query that would be made with `show_query` [[show_query]](https://dplyr.tidyverse.org/reference/explain.html).

In [14]:
beers_db %>%
    filter(ABV > 5) %>%
    summarise(mean = mean(OG)) %>%
    show_query()


<SQL>


Warning message:
“Missing values are always removed in SQL.
Use `mean(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


SELECT AVG(`OG`) AS `mean`
FROM `beer_recipes`
WHERE (`ABV` > 5.0)


One can collect the data to the current R session with `collect` [[collect]](https://dbplyr.tidyverse.org/reference/collapse.tbl_sql.html).

In [15]:
beers_over_5abv <- beers_db %>%
    filter(ABV > 5) %>%
    select(Name, ABV, OG, FG) %>%
    collect()
str(beers_over_5abv)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	85 obs. of  4 variables:
 $ Name: chr  "Vanilla Cream Ale" "Southern Tier Pumking clone" "Zombie Dust Clone - EXTRACT" "Zombie Dust Clone - ALL GRAIN" ...
 $ ABV : num  5.48 8.16 5.91 5.8 6.48 5.58 7.09 5.36 5.77 8.22 ...
 $ OG  : num  1.05 1.08 1.06 1.06 1.06 ...
 $ FG  : num  1.01 1.02 1.02 1.02 1.01 ...


To remove tables from the temporary memory database, one needs to get the connection to it with `src_memdb` and `db_drop_table` from `dplyr`'s SQL backend [[src_memdb]](https://dbplyr.tidyverse.org/rmeference/memdb_frame.html) [[db_drop_table]](https://dplyr.tidyverse.org/reference/backend_dbplyr.html).

In [16]:
memdb <- src_memdb()
memdb
db_drop_table('beer_recipes', con=memdb$con)
memdb

src:  sqlite 3.22.0 [:memory:]
tbls: beer_recipes, sqlite_stat1, sqlite_stat4

[1] 0

src:  sqlite 3.22.0 [:memory:]
tbls: sqlite_stat1, sqlite_stat4

One can also write to a database file by opening a database connection with DBI.

In [17]:
con <- DBI::dbConnect(RSQLite::SQLite(), dbname = "beer_recipes.sqlite")

Let's use `copy_to`-function to copy the `beer_recipes`-dataframe into a table in this newly established SQLite [[copy_to]](https://dplyr.tidyverse.org/reference/copy_to.html).

In [18]:
copy_to(con, beer_recipes, overwrite=TRUE, temporary=FALSE)

To get a reference to this new tab, we can use the `tbl`-function [[tbl]](https://dplyr.tidyverse.org/reference/tbl.html).

In [19]:
beers_sqlite <- tbl(con,'beer_recipes')
beers_sqlite

# Source:   table<beer_recipes> [?? x 22]
# Database: sqlite 3.22.0
#   [/notebooks/python-r-data-analysis-course-dev/r/beer_recipes.sqlite]
   BeerID Name  URL   Style StyleID `Size(L)`    OG    FG   ABV   IBU Color
    <dbl> <chr> <chr> <chr>   <dbl>     <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1      1 Vani… /hom… Crea…      45      21.8  1.06  1.01  5.48  17.6  4.83
 2      2 Sout… /hom… Holi…      85      20.8  1.08  1.02  8.16  60.6 15.6 
 3      3 Zomb… /hom… Amer…       7      18.9  1.06  1.02  5.91  59.2  8.98
 4      4 Zomb… /hom… Amer…       7      22.7  1.06  1.02  5.8   54.5  8.5 
 5      5 Bakk… /hom… Belg…      20      50    1.06  1.01  6.48  17.8  4.57
 6      6 Sier… /hom… Amer…      10      24.6  1.06  1.01  5.58  40.1  8   
 7      7 Russ… /hom… Impe…      86      22.7  1.07  1.02  7.09 269.   6.33
 8      8 Spot… /hom… Crea…      45      20.8  1.05  1.01  5.36  20.0  5.94
 9      9 Choc… /hom… Robu…     129      22.7  1.06  1.02  5.77  31.6 34.8 
10     10 Mang… /hom… I

Let's close and re-open the connection to verify that the table is indeed in the database file.

In [20]:
dbDisconnect(con)
con
con <- DBI::dbConnect(RSQLite::SQLite(), dbname = "beer_recipes.sqlite")
con
tbl(con,'beer_recipes')
dbDisconnect(con)
con

<SQLiteConnection>
  DISCONNECTED

<SQLiteConnection>
  Path: /notebooks/python-r-data-analysis-course-dev/r/beer_recipes.sqlite
  Extensions: TRUE

# Source:   table<beer_recipes> [?? x 22]
# Database: sqlite 3.22.0
#   [/notebooks/python-r-data-analysis-course-dev/r/beer_recipes.sqlite]
   BeerID Name  URL   Style StyleID `Size(L)`    OG    FG   ABV   IBU Color
    <dbl> <chr> <chr> <chr>   <dbl>     <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1      1 Vani… /hom… Crea…      45      21.8  1.06  1.01  5.48  17.6  4.83
 2      2 Sout… /hom… Holi…      85      20.8  1.08  1.02  8.16  60.6 15.6 
 3      3 Zomb… /hom… Amer…       7      18.9  1.06  1.02  5.91  59.2  8.98
 4      4 Zomb… /hom… Amer…       7      22.7  1.06  1.02  5.8   54.5  8.5 
 5      5 Bakk… /hom… Belg…      20      50    1.06  1.01  6.48  17.8  4.57
 6      6 Sier… /hom… Amer…      10      24.6  1.06  1.01  5.58  40.1  8   
 7      7 Russ… /hom… Impe…      86      22.7  1.07  1.02  7.09 269.   6.33
 8      8 Spot… /hom… Crea…      45      20.8  1.05  1.01  5.36  20.0  5.94
 9      9 Choc… /hom… Robu…     129      22.7  1.06  1.02  5.77  31.6 34.8 
10     10 Mang… /hom… I

<SQLiteConnection>
  DISCONNECTED

# Exercise time:

1. Modify column specifications for FIFA World Cup match data [[1]](https://www.kaggle.com/abecklas/fifa-world-cup). Use `col_datetime` in `col_types` to get a good specification for column _DateTime_ [[col_datetime]](http://readr.tidyverse.org/reference/parse_datetime.html). Use `col_factor` to make columns _Round_, _Stadium_, _City_, _HomeTeam_ and _AwayTeam_ into factors.
2. Store the resulting tibble as a feather.
3. Store the resulting tibble into a SQLite database.

In [21]:
fifa_matches <- read_csv("/m/jhnas/jupyter/shareddata/python-r-data-analysis/fifa/WorldCupMatches.csv")

Parsed with column specification:
cols(
  Year = col_double(),
  DateTime = col_character(),
  Round = col_character(),
  Stadium = col_character(),
  City = col_character(),
  HomeTeam = col_character(),
  HomeGoals = col_double(),
  AwayGoals = col_double(),
  AwayTeam = col_character(),
  Observation = col_character()
)



In [22]:
str(fifa_matches)

Classes ‘spec_tbl_df’, ‘tbl_df’, ‘tbl’ and 'data.frame':	4572 obs. of  10 variables:
 $ Year       : num  1930 1930 1930 1930 1930 1930 1930 1930 1930 1930 ...
 $ DateTime   : chr  "13 Jul 1930 - 15:00" "13 Jul 1930 - 15:00" "14 Jul 1930 - 12:45" "14 Jul 1930 - 14:50" ...
 $ Round      : chr  "Group 1" "Group 4" "Group 2" "Group 3" ...
 $ Stadium    : chr  "Pocitos" "Parque Central" "Parque Central" "Pocitos" ...
 $ City       : chr  "Montevideo" "Montevideo" "Montevideo" "Montevideo" ...
 $ HomeTeam   : chr  "France" "USA" "Yugoslavia" "Romania" ...
 $ HomeGoals  : num  4 3 2 3 1 3 4 3 1 1 ...
 $ AwayGoals  : num  1 0 1 1 0 0 0 0 0 0 ...
 $ AwayTeam   : chr  "Mexico" "Belgium" "Brazil" "Peru" ...
 $ Observation: chr  NA NA NA NA ...
 - attr(*, "spec")=
  .. cols(
  ..   Year = col_double(),
  ..   DateTime = col_character(),
  ..   Round = col_character(),
  ..   Stadium = col_character(),
  ..   City = col_character(),
  ..   HomeTeam = col_character(),
  ..   HomeGoals = col_double(